In [1]:
%matplotlib inline

from boututils.datafile import DataFile
from boutdata.collect import collect
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, sys, pathlib
import platform
import traceback
import xarray as xr
import xbout
import scipy
import re

onedrive_path = onedrive_path = str(os.getcwd()).split("OneDrive")[0] + "OneDrive"
sys.path.append(os.path.join(onedrive_path, r"Project\python-packages\sdtools"))
sys.path.append(os.path.join(onedrive_path, r"Project\python-packages\soledge"))
sys.path.append(os.path.join(onedrive_path, r"Project\python-packages"))


# from gridtools.hypnotoad_tools import *
from gridtools.b2_tools import *
from gridtools.utils import *

from hermes3.case_db import *
from hermes3.load import *
from hermes3.named_selections import *
from hermes3.plotting import *
from hermes3.grid_fields import *
from hermes3.accessors import *
from hermes3.utils import *
from code_comparison.code_comparison import *

import gridtools.solps_python_scripts.setup
from gridtools.solps_python_scripts.plot_solps       import plot_1d, plot_2d
import ipywidgets as widgets

%load_ext autoreload
%autoreload 2


print("Done")


st40_phys_viewer can not be loaded from outside Tokamak Energy!

Done


## Tallies
- I need to have a SOLPS install to use this utility
- Instead, Matteo copy pasted the output of the tool into a txt file
- Here I am parsing it in Pandas.
- Note that the numbers in the columns correspond to region numbers - see SOLPS manual.

In [2]:
df = pd.read_csv(os.path.join(onedrive_path, r"Project\collab\From\Matteo\2023-03-08 - Display tallies ppt\display_tallies.csv"), skiprows=1, names = range(14+1))
df = df[df.index.notnull()].reset_index().rename({"index" : "param"}, axis = 1)
df[df["param"].str.contains("E\+")] = df[df["param"].str.contains("E\+")].shift(axis=1)


params = df["param"][df["param"].str.contains("D0|D\+|None") == False].values

for i in df.index:
    if df.loc[i, "param"] == None:
        df.loc[i, "param"] = df.loc[i-1, "param"]
        df.loc[i-1, "param"] = None

    if df.loc[i, "param"] == "D0":
        name = df.loc[i-1, "param"]
        df.loc[i, "param"] = name+"_n"
        df.loc[i+1, "param"] = name+"_i"
        df.loc[i-1] = df.loc[i] + df.loc[i+1]   # The name without suffix becomes sum of neutrals and ions
        df.loc[i-1, "param"] = name
        
df = df[~df["param"].isna()]

for i in range(15):
    df[i] = df[i].astype(float)
df.head(10)

,param,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,rsanareg,1.110000e+14,4.350000e+13,1.130000e+12,3.280000e+11,3.300000e+11,6.100000e+13,3.710000e+12,6.460000e+11,6.400000e+11,NaN,NaN,NaN,NaN,NaN,NaN
1,rsanareg_n,1.110000e+14,4.350000e+13,1.130000e+12,3.280000e+11,3.300000e+11,6.100000e+13,3.710000e+12,6.460000e+11,6.400000e+11,NaN,NaN,NaN,NaN,NaN,NaN
2,rsanareg_i,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN
3,rsahireg,6.340000e-06,5.650000e-08,2.340000e-07,2.810000e-07,2.860000e-07,1.650000e-07,7.510000e-07,2.290000e-06,2.280000e-06,NaN,NaN,NaN,NaN,NaN,NaN
4,rsahireg_n,6.340000e-06,5.650000e-08,2.340000e-07,2.810000e-07,2.860000e-07,1.650000e-07,7.510000e-07,2.290000e-06,2.280000e-06,NaN,NaN,NaN,NaN,NaN,NaN
5,rsahireg_i,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN
6,rsamoreg,-2.810000e-11,-1.040000e-11,-2.300000e-13,5.220000e-12,-5.330000e-12,-1.570000e-11,-1.750000e-12,2.450000e-11,-2.440000e-11,NaN,NaN,NaN,NaN,NaN,NaN
7,rsamoreg_n,-2.810000e-11,-1.040000e-11,-2.300000e-13,5.220000e-12,-5.330000e-12,-1.570000e-11,-1.750000e-12,2.450000e-11,-2.440000e-11,NaN,NaN,NaN,NaN,NaN,NaN
8,rsamoreg_i,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN
9,rranareg,1.150000e+08,4.190000e+07,2.170000e+06,8.240000e+05,8.200000e+05,5.890000e+07,8.040000e+06,1.080000e+06,1.070000e+06,NaN,NaN,NaN,NaN,NaN,NaN


## Regions

In [3]:
dfreg = df[((df["param"].str.contains("x|y") == False) & (df["param"].str.contains("reg") == True))]

dfreg = dfreg.rename({
    0 : "total",
    1 : "inner_core",
    2 : "inner_sol",
    3 : "inner_lower_target",
    4 : "inner_upper_target",
    5 : "outer_core",
    6 : "outer_sol",
    7 : "outer_upper_target",
    8 : "outer_lower_target"
}, axis = 1
)
dfreg = dfreg.dropna(axis=1)

dfreg.head()

,param,total,inner_core,inner_sol,inner_lower_target,inner_upper_target,outer_core,outer_sol,outer_upper_target,outer_lower_target
0,rsanareg,1.110000e+14,4.350000e+13,1.130000e+12,3.280000e+11,3.300000e+11,6.100000e+13,3.710000e+12,6.460000e+11,6.400000e+11
1,rsanareg_n,1.110000e+14,4.350000e+13,1.130000e+12,3.280000e+11,3.300000e+11,6.100000e+13,3.710000e+12,6.460000e+11,6.400000e+11
2,rsanareg_i,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,rsahireg,6.340000e-06,5.650000e-08,2.340000e-07,2.810000e-07,2.860000e-07,1.650000e-07,7.510000e-07,2.290000e-06,2.280000e-06
4,rsahireg_n,6.340000e-06,5.650000e-08,2.340000e-07,2.810000e-07,2.860000e-07,1.650000e-07,7.510000e-07,2.290000e-06,2.280000e-06


In [4]:
dfxreg = df[((df["param"].str.contains("xreg") == True))]

dfxreg = dfxreg.rename({
    0 : "total",
    1 : "inner_lower_target",
    2 : "inner_lower_entrance",
    3 : "inner_upper_entrance",
    4 : "inner_upper_target",
    5 : "outer_upper_target",
    6 : "outer_upper_entrance",
    7 : "outer_lower_entrance",
    8 : "outer_lower_target",
    9 : "lower_inner_pfr_to_inner_core",
    10 : "inner_core_to_inner_upper_pfr",
    11 : "outer_upper_pfr_to_outer_core",
    12 : "outer_core_to_outer_lower_pfr"
}, axis = 1
)

dfxreg

,param,total,inner_lower_target,inner_lower_entrance,inner_upper_entrance,inner_upper_target,outer_upper_target,outer_upper_entrance,outer_lower_entrance,outer_lower_target,lower_inner_pfr_to_inner_core,inner_core_to_inner_upper_pfr,outer_upper_pfr_to_outer_core,outer_core_to_outer_lower_pfr,13,14
30,fnaxreg,0.0,-1.860000e+21,-1.030000e+21,1.050000e+21,1.890000e+21,-6.540000e+21,-3.770000e+21,3.780000e+21,6.530000e+21,-1.980000e+20,-1.390000e+20,5.080000e+20,1.400000e+20,NaN,NaN
31,fnaxreg_n,0.0,1.860000e+11,-5.250000e+10,5.100000e+10,-1.890000e+11,6.540000e+11,-9.250000e+10,9.610000e+10,-6.530000e+11,8.240000e+08,9.230000e+09,-4.240000e+08,-6.780000e+09,NaN,NaN
32,fnaxreg_i,0.0,-1.860000e+21,-1.030000e+21,1.050000e+21,1.890000e+21,-6.540000e+21,-3.770000e+21,3.780000e+21,6.530000e+21,-1.980000e+20,-1.390000e+20,5.080000e+20,1.400000e+20,NaN,NaN
37,fhixreg,0.0,-5.630000e+04,-7.910000e+04,8.000000e+04,5.730000e+04,-1.840000e+05,-2.670000e+05,2.660000e+05,1.840000e+05,-6.350000e+04,-1.230000e+03,1.410000e+05,1.510000e+03,NaN,NaN
41,fhexreg,0.0,-6.190000e+04,-7.880000e+04,7.940000e+04,6.250000e+04,-2.070000e+05,-2.630000e+05,2.620000e+05,2.070000e+05,-3.860000e+04,2.380000e+03,9.930000e+04,-2.230000e+03,NaN,NaN
44,fhpxreg,0.0,-4.060000e+03,-2.250000e+03,2.290000e+03,4.120000e+03,-1.430000e+04,-8.210000e+03,8.230000e+03,1.420000e+04,-4.310000e+02,-3.040000e+02,1.110000e+03,3.050000e+02,NaN,NaN
45,fhpxreg_n,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN
46,fhpxreg_i,0.0,-4.060000e+03,-2.250000e+03,2.290000e+03,4.120000e+03,-1.430000e+04,-8.210000e+03,8.230000e+03,1.420000e+04,-4.310000e+02,-3.040000e+02,1.110000e+03,3.050000e+02,NaN,NaN
50,fhmxreg,0.0,-2.230000e+03,4.370000e+03,-4.360000e+03,2.130000e+03,-1.340000e+04,2.520000e+04,-2.530000e+04,1.340000e+04,-4.540000e-01,1.270000e+03,1.680000e-01,-1.280000e+03,NaN,NaN
51,fhmxreg_n,0.0,-2.060000e-10,-2.890000e-08,2.950000e-08,1.540000e-10,-3.500000e-09,-1.160000e-07,1.170000e-07,4.160000e-09,1.200000e-08,3.310000e-08,-1.190000e-08,-3.420000e-08,NaN,NaN


In [23]:
dfyreg = df[((df["param"].str.contains("yreg") == True))]

dfyreg = dfyreg.rename({
    0 : "total",
    1 : "inner_lower_pfr",
    2 : "inner_core",
    3 : "inner_upper_pfr",
    4 : "inner_separatrix",
    5 : "inner_lower_sol",
    6 : "inner_sol",
    7 : "inner_upper_sol",
    8 : "outer_upper_pfr",
    9 : "outer_core",
    10 : "outer_lower_pfr",
    11 : "outer_separatrix",
    12 : "outer_upper_sol",
    13 : "outer_sol",
    14 : "outer_lower_sol"
}, axis = 1
)

dfyreg

,param,total,inner_lower_pfr,inner_core,inner_upper_pfr,inner_separatrix,inner_lower_sol,inner_sol,inner_upper_sol,outer_upper_pfr,outer_core,outer_lower_pfr,outer_separatrix,outer_upper_sol,outer_sol,outer_lower_sol
33,fnayreg,0.0,-1.460002e+16,2.490000e+20,-8.610019e+15,1.450000e+21,4.550011e+15,6.460019e+16,4.250011e+15,-3.690006e+16,3.350000e+20,-6.170008e+16,4.740000e+21,1.750000e+18,2.650000e+19,1.740000e+18
34,fnayreg_n,0.0,-2.470000e+10,1.750000e+12,-1.870000e+10,1.960000e+12,1.100000e+10,1.890000e+11,1.110000e+10,-6.180000e+10,4.760000e+12,-8.230000e+10,5.960000e+12,1.610000e+11,1.360000e+12,1.600000e+11
35,fnayreg_i,0.0,-1.460000e+16,2.490000e+20,-8.610000e+15,1.450000e+21,4.550000e+15,6.460000e+16,4.250000e+15,-3.690000e+16,3.350000e+20,-6.170000e+16,4.740000e+21,1.750000e+18,2.650000e+19,1.740000e+18
39,fhiyreg,0.0,-7.110000e-01,3.260000e+05,-4.250000e-01,1.820000e+05,2.640000e-01,4.190000e+00,2.470000e-01,-2.520000e+00,6.750000e+05,-4.220000e+00,6.230000e+05,5.840000e+01,1.100000e+03,5.800000e+01
43,fheyreg,0.0,-1.600000e-02,2.530000e+05,-8.700000e-03,1.750000e+05,4.900000e-03,7.850000e-02,4.560000e-03,-4.010000e-02,5.080000e+05,-7.220000e-02,6.040000e+05,3.610000e+00,6.770000e+01,3.590000e+00
47,fhpyreg,0.0,-3.180000e-02,5.430000e+02,-1.880000e-02,3.160000e+03,9.920000e-03,1.410000e-01,9.260000e-03,-8.040000e-02,7.300000e+02,-1.340000e-01,1.030000e+04,3.820000e+00,5.770000e+01,3.800000e+00
48,fhpyreg_n,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
49,fhpyreg_i,0.0,-3.180000e-02,5.430000e+02,-1.880000e-02,3.160000e+03,9.920000e-03,1.410000e-01,9.260000e-03,-8.040000e-02,7.300000e+02,-1.340000e-01,1.030000e+04,3.820000e+00,5.770000e+01,3.800000e+00
53,fhmyreg,0.0,-8.450000e-03,1.160000e-03,-5.350000e-03,-2.500000e+02,1.240000e-02,6.720000e-02,1.350000e-02,-1.710000e-02,1.250000e-03,-2.770000e-02,-4.700000e+02,4.290000e+00,8.010000e+00,4.270000e+00
54,fhmyreg_n,0.0,-2.990000e-10,1.310000e-11,-7.370000e-11,-2.230000e-07,9.150000e-11,3.890000e-09,1.330000e-10,-7.160000e-10,2.220000e-10,-1.520000e-09,-1.510000e-06,1.960000e-09,8.440000e-08,1.910000e-09


In [39]:
dfyreg = df[((df["param"].str.contains("yreg") == True))]

dfyreg = dfyreg.rename({
    0 : "total",
    1 : "inner_lower_pfr",
    2 : "inner_core",
    3 : "inner_upper_pfr",
    4 : "inner_separatrix",
    5 : "inner_lower_sol",
    6 : "inner_sol",
    7 : "inner_upper_sol",
    8 : "outer_upper_pfr",
    9 : "outer_core",
    10 : "outer_lower_pfr",
    11 : "outer_separatrix",
    12 : "outer_upper_sol",
    13 : "outer_sol",
    14 : "outer_lower_sol"
}, axis = 1
)

dfyreg

0

In [27]:
def read_display_tallies(path):
    """ 
    This reads a csv file coming from a copy paste of the output of the display_tallies SOLPS routine
    It parses the table into three dataframes corresponding to the volumetric tallies,
    and the tallies of fluxes between regions in the x and y directions
    Each variable comes in three flavours: suffixless = total, _n = neutrals, _i = ions
    """
    
    # Read display tallies csv copied from console
    df = pd.read_csv(path, skiprows=1, names = range(14+1))
    df = df[df.index.notnull()].reset_index().rename({"index" : "param"}, axis = 1)
    
    # Some lines have no parameter title and the delimiter parsing makes it start as a number
    # Shift those back to be in line with the others
    df[df["param"].str.contains("E\+")] = df[df["param"].str.contains("E\+")].shift(axis=1)

    params = df["param"][df["param"].str.contains("D0|D\+|None") == False].values

    for i in df.index:
        # Empty rows due to the shifting above are now labelled correctly
        if df.loc[i, "param"] == None:
            df.loc[i, "param"] = df.loc[i-1, "param"]
            df.loc[i-1, "param"] = None

        # Label D0 as _n, D+1 as _i and D0 + D+1 as the original parameter
        if df.loc[i, "param"] == "D0":
            name = df.loc[i-1, "param"]
            df.loc[i, "param"] = name+"_n"
            df.loc[i+1, "param"] = name+"_i"
            df.loc[i-1] = df.loc[i] + df.loc[i+1]   # The name without suffix becomes sum of neutrals and ions
            df.loc[i-1, "param"] = name
            
    # Remove all None junk
    df = df[~df["param"].isna()]

    for i in range(15):
        df[i] = df[i].astype(float)

    # Parse the volumetric regions
    dfreg = df[((df["param"].str.contains("x|y") == False) & (df["param"].str.contains("reg") == True))]
    dfreg = dfreg.rename({
        0 : "total",
        1 : "inner_core",
        2 : "inner_sol",
        3 : "inner_lower_target",
        4 : "inner_upper_target",
        5 : "outer_core",
        6 : "outer_sol",
        7 : "outer_upper_target",
        8 : "outer_lower_target"
    }, axis = 1
    )
    dfreg = dfreg.dropna(axis=1)

    # Parse the fluxes between regions in the X direction
    dfxreg = df[((df["param"].str.contains("xreg") == True))]
    dfxreg = dfxreg.rename({
        0 : "total",
        1 : "inner_lower_target",
        2 : "inner_lower_entrance",
        3 : "inner_upper_entrance",
        4 : "inner_upper_target",
        5 : "outer_upper_target",
        6 : "outer_upper_entrance",
        7 : "outer_lower_entrance",
        8 : "outer_lower_target",
        9 : "lower_inner_pfr_to_inner_core",
        10 : "inner_core_to_inner_upper_pfr",
        11 : "outer_upper_pfr_to_outer_core",
        12 : "outer_core_to_outer_lower_pfr"
    }, axis = 1
    )
    
    # Parse the fluxes between regions in the Y direction
    dfyreg = df[((df["param"].str.contains("yreg") == True))]
    dfyreg = dfyreg.rename({
        0 : "total",
        1 : "inner_lower_pfr",
        2 : "inner_core",
        3 : "inner_upper_pfr",
        4 : "inner_separatrix",
        5 : "inner_lower_sol",
        6 : "inner_sol",
        7 : "inner_upper_sol",
        8 : "outer_upper_pfr",
        9 : "outer_core",
        10 : "outer_lower_pfr",
        11 : "outer_separatrix",
        12 : "outer_upper_sol",
        13 : "outer_sol",
        14 : "outer_lower_sol"
    }, axis = 1
    )

    return {"reg":dfreg, "xreg":dfxreg, "yreg":dfyreg }

In [28]:
path = os.path.join(onedrive_path, r"Project\collab\From\Matteo\2023-03-08 - Display tallies ppt\display_tallies.csv")

read_display_tallies(path)

{'reg':                 param         total    inner_core     inner_sol  \
 0            rsanareg  1.110000e+14  4.350000e+13  1.130000e+12   
 1          rsanareg_n  1.110000e+14  4.350000e+13  1.130000e+12   
 2          rsanareg_i  0.000000e+00  0.000000e+00  0.000000e+00   
 3            rsahireg  6.340000e-06  5.650000e-08  2.340000e-07   
 4          rsahireg_n  6.340000e-06  5.650000e-08  2.340000e-07   
 5          rsahireg_i  0.000000e+00  0.000000e+00  0.000000e+00   
 6            rsamoreg -2.810000e-11 -1.040000e-11 -2.300000e-13   
 7          rsamoreg_n -2.810000e-11 -1.040000e-11 -2.300000e-13   
 8          rsamoreg_i  0.000000e+00  0.000000e+00  0.000000e+00   
 9            rranareg  1.150000e+08  4.190000e+07  2.170000e+06   
 10         rranareg_n  0.000000e+00  0.000000e+00  0.000000e+00   
 11         rranareg_i  1.150000e+08  4.190000e+07  2.170000e+06   
 12           rrahireg  7.770000e-12  6.250000e-14  4.360000e-13   
 13         rrahireg_n  0.000000e+00  0.0